In [ ]:
import os
from typing import Literal
from deepagents import create_deep_agent
from langchain_openai import ChatOpenAI
from openai import APIStatusError
import logging

# System prompt to steer the agent to be an expert researcher
research_instructions = """You are an expert researcher. Your job is to conduct thorough research, and then write a polished report."""

In [22]:
model_glm_4_5 = ChatOpenAI(
  api_key=os.getenv("OPENROUTER_API_KEY"),
  base_url=os.getenv("OPENROUTER_BASE_URL"),
  model="z-ai/glm-4.5-air:free",
  default_headers={
    "HTTP-Referer": "https://www.your-site.com",
    "X-Title": "Your Site",
  }
)


model_gpt_5 = ChatOpenAI(
    api_key=os.getenv("AGENTROUTER_API_KEY"),
    base_url="https://agentrouter.org/v1",
    model="gpt-5",
    default_headers={
        "HTTP-Referer": "https://github.com/RooVetGit/Roo-Cline",
        "X-Title": "Roo Code",
        "User-Agent": "RooCode/1.0.0",
    },
)

models = [model_glm_4_5, model_gpt_5]

In [23]:
def invoke_with_fallback(models_to_try, user_prompt, system_prompt):
    """
    Calls the deep agent with a list of models, switching to the next model in case of HTTP errors.
    """
    for i, model in enumerate(models_to_try):
        print(f"--- Попытка {i + 1}/{len(models_to_try)} с моделью: {model.model_name} ---")
        try:
            agent = create_deep_agent(
                model=model,
                system_prompt=system_prompt,
            )
            result = agent.invoke({"messages": [{"role": "user", "content": user_prompt}]})
            print(f"--- Успешный ответ от модели: {model.model_name} ---")
            return result
        except APIStatusError as e:
            if 400 <= e.status_code < 600:
                logging.warning(f"Модель {model.model_name} завершилась с ошибкой HTTP {e.status_code}. Пробуем следующую модель. Ошибка: {e}")
                continue
            else:
                logging.error(f"Модель {model.model_name} завершилась с неисправимой ошибкой API: {e}")
                raise e
        except Exception as e:
            logging.error(f"Произошла непредвиденная ошибка с моделью {model.model_name}: {e}. Пробуем следующую модель.")
            continue
        
    return {"messages": [{"role": "assistant", "content": "Не удалось получить ответ ни от одной из доступных моделей."}]}

In [ ]:
# user_task = "Создай мне тренировку."
# result = invoke_with_fallback(
#     models_to_try=models,
#     user_prompt=user_task,
#     system_prompt=research_instructions
# )

--- Попытка 1/2 с моделью: z-ai/glm-4.5-air:free ---
--- Успешный ответ от модели: z-ai/glm-4.5-air:free ---


In [28]:
# from IPython.display import Image, display

# # Create the deep agent
# agent = create_deep_agent(
#     model=models.get("gpt-5"),
#     # tools=[internet_search],
#     system_prompt=research_instructions,
# )

# display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [29]:
# Invoke the agent
# result = agent.invoke({"messages": [{"role": "user", "content": "Создай мне тренировку."}]})

In [ ]:
def process_user_message(user_prompt: str):
    """
        Processes a user prompt by invoking a series of language models with fallback logic.

        Args:
            user_prompt: The user's message.

        Returns:
            The content of the assistant's final message as a string.


    """
    result = invoke_with_fallback(
        models_to_try=models,
        user_prompt=user_prompt,
        system_prompt=research_instructions
    )
    
    if result and "messages" in result and result["messages"]:
        last_message = result["messages"][-1]
        if hasattr(last_message, 'content'):
            return last_message.content
        elif isinstance(last_message, dict) and 'content' in last_message:
            return last_message['content']
        
        return "Агент не смог обработать ваш запрос."


In [ ]:
# from utils.utils import format_messages

# format_messages(result["messages"])



╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Создай мне тренировку.                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Я создам для вас комплексную тренировку. Давайте составлю сбалансированную программу, которая включает          │
│ разминку, основные упражнения и заминку.                                                                        │
│                                                                                                                 │
│                                                                                                                 │
│ 🔧 Tool Call: write_file                                                                                        │
│    Args: {                                                                                                      │
│   "file_path": "/workout_routine.txt",                                                                          │
│   "content": "КОМПЛЕКСНАЯ ТРЕНИРОВКА НА ВЕС ТЕЛА\n\nРАЗМИНКА (10-15 минут)\n1. Бег на месте - 2 минуты\n2.      │
│ Ходьба с высоким подниманием колен - 1 минута\n3. Вращение руками вперед и назад - по 30 секунд\n4. Вращение    │
│ стопами - по 30 секунд\n5. Наклоны в стороны - 10 раз\n6. Повороты корпуса - 10 раз в каждую сторону\n7.        │
│ Круговые движения тазом - по 10 раз в каждую сторону\n8. Прыжки на месте - 30 секунд\n\nОСНОВНАЯ ЧАСТЬ (30-40   │
│ минут)\n\nСИЛОВАЯ ПОДГОТОВКА:\n1. Отжимания (на коленях или классические) - 3 подхода по 10-15 повторений\n2.   │
│ Приседания - 3 подхода по 15-20 повторений\n3. Планка - 3 подхода по 30-60 секунд\n4. Ягодичный мостик - 3      │
│ подхода по 15-20 повторений\n5. Выпады вперед - 3 подхода по 10-12 повторений на каждую ногу\n6. Подтягивания   │
│ (если есть турник) или тяга резины - 3 подхода по 8-12 повторений\n\nКАРДИО-НАГРУЗКА:\n1. Бурпи - 3 подхода по  │
│ 8-10 повторений\n2. Прыжки \"скакалкой\" - 3 подхода по 1-2 минуты\n3. Бег с высоким подниманием бедер - 3      │
│ подхода по 30 секунд\n4. \"Горка\" (Mountain climbers) - 3 подхода по 20-30 повторений\n\nГИБКОСТЬ И            │
│ СТАБИЛИЗАЦИЯ:\n1. Кошка-корова - 3 подхода по 10-15 повторений\n2. Поза ребенка - 3 подхода по 30 секунд\n3.    │
│ Растяжка квадрицепса - 3 подхода по 30 секунд на каждую ногу\n4. Растяжка подколенных сухожилий - 3 подхода по  │
│ 30 секунд на каждую ногу\n5. Растяжка груди у стены - 3 подхода по 30 секунд\n6. Шея уши - по 10 повторений в   │
│ каждую сторону\n\nЗАМИНКА (10 минут)\n1. Медленная ходьба - 2 минуты\n2. Растяжка основных мышечных групп:\n    │
│ - Растяжка бедер (поза голубя) - 30 секунд на каждую сторону\n   - Растяжка спины (складка) - 30 секунд\n   -   │
│ Растяжка груди - 30 секунд\n   - Растяжка трицепса - 30 секунд на каждую руку\n   - Растяжка поясницы - 30      │
│ секунд\n3. Глубокое дыхание - 2 минуты\n\nРЕКОМЕНДАЦИИ:\n- Начинайте с легкой интенсивности и постепенно        │
│ увеличивайте нагрузку\n- Следите за техникой выполнения упражнений\n- Отдыхайте между подходами 30-60 секунд\n- │
│ Пейте воду во время тренировки\n- Тренируйтесь 3-4 раза в неделю\n- При дискомфорте или боли немедленно         │
│ прекратите упражнение\n\nПРИМЕР РАСПИСАНИЯ ПО НЕДЕЛЯМ:\nПонедельник: Силовая тренировка\nВторник: Отдых или     │
│ легкая кардио нагрузка\nСреда: Кардио тренировка\nЧетверг: Отдых\nПятница: Силовая тренировка\nСуббота:         │
│ Гибкость и растяжка\nВоскресенье: Отдых"                                                                        │
│ }                                                                                                               │
│    ID: call_ENps1rECSSuOhcSfoBlA8g                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Updated file /workout_routine.txt                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ Отлично! Я создал для вас комплексную тренировку по адресу `/workout_routine.txt`.                              │
│                                                                                                                 │
│ **Что включает программа:**                                                                                     │
│ - **Разминка** (10-15 минут) - подготовка организма к нагрузке                                                  │
│ - **Основная часть** (30-40 минут) - силовые упражнения, кардио и гибкость                                      │
│ - **Заминка** (10 минут) - растяжка и восстановление                                                            │
│                                                                                                                 │
│ **Особенности тренировки:**                                                                                     │
│ - Упражнения выполняются с собственным весом, не требует оборудования                                           │
│ - Подходит для разных уровней подготовки (есть модификации)                                                     │
│ - Сбалансированное развитие всех мышечных групп                                                                 │
│ - Рекомендованное расписание на неделю                                                                          │
│                                                                                                                 │
│ **Рекомендации по использованию:**                                                                              │
│ - Начинайте с легкой интенсивности и постепенно увеличивайте нагрузку                                           │
│ - Сосредоточьтесь на технике выполнения упражнений                                                              │
│ - Тренируйтесь 3-4 раза в неделю с отдыхом между днями                                                          │
│ - Пейте достаточно воды во время тренировки                                                                     │
│ - При дискомфорте немедленно прекращайте упражнение                                                             │
│                                                                                                                 │
│ Вы можете открыть файл и начать тренировку прямо сейчас. Если нужно адаптировать программу под ваши конкретные  │
│ цели или уровень подготовки, дайте мне знать!                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [21]:
result

{'messages': [HumanMessage(content='Создай мне тренировку.', additional_kwargs={}, response_metadata={}, id='acf7725b-02c3-41ca-8585-99f10e0eab22'),
  AIMessage(content='\n\n', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 358, 'prompt_tokens': 4369, 'total_tokens': 4727, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'z-ai/glm-4.5-air:free', 'system_fingerprint': None, 'id': 'gen-1761129290-qEhZE0ASXBVFHWMKKeb0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--41c93769-6fed-4835-9b25-d3c5fbfdfb50-0', invalid_tool_calls=[{'type': 'invalid_tool_call', 'id': 'call_f50179e40da54fbda42fe36b', 'name': 'write_todos', 'args': '{"todos": [{"content": "\\u0423\\u0442\\u043e\\u0447\\u043d\\u0438\\u0442\\u044c \\u0446\\u0435\\u043b\\u0438 \\u043f\\u043e\\u043b\\u044c\\u0437\\u043e\\u0432\\u0430\\u0442\\u0435\\u043b\\u044f (\\u043d\\u0430\\u0431\\u043e\\u0440 \\u043c\\u044b\\u